In [47]:
from __future__ import division
import csv,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time, locale
from IPython import display as d
import nvd3

reload(sys)  # Reload does the trick!
sys.setdefaultencoding('UTF8')

nvd3.ipynb.initialize_javascript(use_remote=True)

locale.setlocale(locale.LC_ALL, 'es_ES')
locale.setlocale(locale.LC_ALL, 'C')
locale.format("%10.2f", 1234.567, grouping=True) 
pd.options.display.float_format = '{:,.2f}'.format

# Static files
FILE_SUBVENCIONES = 'data/datos_subvenciones.csv'
FILE_AYUDAS = 'data/ayudas.json'
FILE_ELECCIONES = 'data/04_201105_1.csv'
FILE_MUNICIPIOS = 'data/smex99.csv'
# Referencias de partidas del presupuesto: http://www.juntadeandalucia.es/haciendayadministracionpublica/planif_presup/proy_presupuesto2014/presentacion/presentacion-5.pdf


def totimestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    ttime = (td.microseconds + (td.seconds + td.days * 86400) * 10**6) / 10**6 
    return ttime * 1000

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
#Read files
subvenciones = pd.read_csv(FILE_SUBVENCIONES, decimal=',')
ayudas = pd.read_json(FILE_AYUDAS)
municipios = pd.read_csv(FILE_MUNICIPIOS, quotechar='"', sep=';', decimal=',', thousands='.')
elecciones = pd.read_csv(FILE_ELECCIONES, quotechar='"', sep=';', decimal=',', thousands='.')

El importe total de las subvenciones includas en los datos abiertos es:

In [49]:
subvenciones['importe'].sum()

1028817129.8299986

In [50]:
subvenciones['mes_concesion'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y%m'))
subvenciones['fecha_concesion_tmp'] = subvenciones['fecha_concesion'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
subvenciones['mes_concesion_timestamp'] = subvenciones['fecha_concesion'].apply(lambda x: totimestamp(datetime(datetime.fromtimestamp(x).year, datetime.fromtimestamp(x).month,1,0,0)))
subvenciones['grupo_pres'] = subvenciones['programa'].apply(lambda x: ("%s" % x)[0])

In [51]:
subvenciones_por_fecha = subvenciones.groupby('fecha_concesion').sum()
subvenciones_por_mes = subvenciones.groupby('mes_concesion_timestamp').sum()
subvenciones_por_mes_grupo = subvenciones.groupby(['mes_concesion_timestamp','grupo_pres']).sum().reset_index()

In [52]:
type = 'stackedAreaChart'
chart2 = nvd3.stackedAreaChart(name=type,height=450,width=850,use_interactive_guideline=True, x_is_date=True)
#nb_element = 50
#xdata = range(len(list(subvenciones_por_mes.index)))
xdata = list(subvenciones_por_mes.index)
ydata = list(subvenciones_por_mes['importe'])
chart2.add_serie(name="Ejecución Subvenciones", y=ydata, x=xdata)
chart2.margin_left = 100
chart2

In [53]:
type = 'pieChart'
chart1 = nvd3.pieChart(name=type, color_category='category20c', height=450, width=450)
chart1.set_containerheader("\n\n<h2>" + type + "</h2>\n\n")

#Create the keys
xdata = ["Orange", "Banana", "Pear", "Kiwi", "Apple", "Strawberry", "Pineapple"]
ydata = [3, 4, 0, 1, 5, 7, 3]

#Add the serie
extra_serie = {"tooltip": {"y_start": "", "y_end": " cal"}}
chart1.add_serie(y=ydata, x=xdata, extra=extra_serie)
chart1

In [79]:
chart3 = nvd3.stackedAreaChart(name="Subvenciones por Área",height=450,width=850,use_interactive_guideline=True, x_is_date=True)
xdata = list(subvenciones_por_mes_grupo.mes_concesion_timestamp)
ydata_1 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '1']['importe'])
ydata_2 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '2']['importe'])
ydata_3 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '3']['importe'])
ydata_4 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '4']['importe'])
ydata_5 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '5']['importe'])
ydata_6 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '6']['importe'])
ydata_7 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '7']['importe'])
ydata_8 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '8']['importe'])
ydata_9 = list(subvenciones_por_mes_grupo[subvenciones_por_mes_finalidad.grupo_pres == '9']['importe'])
#chart3.add_serie(name="Srv. Generales", y=ydata_1, x=xdata)
#chart3.add_serie(name="Srv. Generales 2", y=ydata_2, x=xdata)
chart3.add_serie(name="Srv. Generales 3", y=ydata_3, x=xdata)
#chart3.add_serie(name="Srv. Generales", y=ydata_4, x=xdata)
#chart3.add_serie(name="Srv. Generales", y=ydata_5, x=xdata)
#chart3.add_serie(name="Srv. Generales", y=ydata_6, x=xdata)
#chart3.add_serie(name="Srv. Generales", y=ydata_7, x=xdata)
chart3.add_serie(name="Srv. Generales 8", y=ydata_8, x=xdata)
#chart3.add_serie(name="Srv. Generales 9", y=ydata_9, x=xdata)
chart3.margin_left = 100

[11146019.829999993,
 1492566.3899999997,
 3176851.8699999982,
 5480667.4800000014,
 3291973.569999998,
 5533100.9200000009,
 8842959.929999996,
 4159566.5899999994,
 182916599.21999985,
 94617290.699999928,
 14292492.689999999,
 50871926.70000004,
 2603657.2400000002,
 159368.06,
 7974428.5799999991]

In [76]:
elecciones['Nombre de Comunidad'] = elecciones['Nombre de Comunidad'].apply(lambda x: x.rstrip())
filas_andalucia = elecciones['Nombre de Comunidad'] == 'Andaluc\xc3\xada' #Andalucía
#cols_elecciones_significativas = (elecciones[rows_andalucia].sum() > 1000)
#resumen_elecciones_andalucia = elecciones[filas_andalucia][elecciones.columns[cols_elecciones_significativas]]

In [16]:
subvenciones_por_finalidad = subvenciones.groupby('finalidad').sum().sort('importe',ascending=0).reset_index()